In [1]:
import math

#===================================================================================================
# INPUTS
#===================================================================================================

K = 8.314472 * 0.001  # Gas constant in kJ/mol/K    kJ/mol/K
V = 1660              # average volume per molecule for a reference concentration of  is 1.661 x 10^3 Å^3/molecule.

T      = 300.0          # Temperature in Kelvin

r0      = 0       # Distance in A
thA_0   = 0     # Angle in degrees (protein)
thB_0   = 0     # Angle in degrees (ligand)

#  kcal * 4.186 ---->  kJ

# BORESCH FORMULA
K_r    = 20 * 4.186      # force constant for distance (kJ/mol/A^2)  
K_thA  = 200 * 4.186     # force constant for angle    (kJ/mol/rad^2)
K_thB  = 200 * 4.186     # force constant for angle    (kJ/mol/rad^2)
K_phiA = 200 * 4.186     # force constant for dihedral (kJ/mol/rad^2)
K_phiB = 200 * 4.186     # force constant for dihedral (kJ/mol/rad^2)
K_phiC = 200 * 4.186     # force constant for dihedral (kJ/mol/rad^2)

# ROUX FORMULA
K_dist    = 20 * 4.186     # force constant for distance (kJ/mol/A^2)  
K_ang     = 200 * 4.186    # force constant for angle    (kJ/mol/rad^2)
K_rotate  = 200 * 4.186    # force constant for angle    (kJ/mol/rad^2)

In [ ]:
with open('./init_structure/Boresch_restraint.tmpl','r') as f:
    lines = [line for line in f.readlines() if line.strip()]
    r0 = float(lines[1].split()[1].split('=')[1].strip(','))
    thA_0 = float(lines[5].split()[1].split('=')[1].strip(','))
    thB_0 = float(lines[9].split()[1].split('=')[1].strip(','))

In [2]:
#===================================================================================================
# BORESCH FORMULA
#===================================================================================================

thA = math.radians(thA_0)  # convert angle from degrees to radians --> math.sin() wants radians
thB = math.radians(thB_0)  # convert angle from degrees to radians --> math.sin() wants radians

arg =(
    (8.0 * math.pi**2.0 * V) / (r0**2.0 * math.sin(thA) * math.sin(thB)) 
    * 
    (
        ( (K_r * K_thA * K_thB * K_phiA * K_phiB * K_phiC)**0.5 ) / ( (2.0 * math.pi * K * T)**(3.0) )
    )
)

In [3]:
dG = - K * T * math.log(arg)

print ("dG_off = %8.3f kcal/mol" %(dG/4.186))
print ("dG_on  = %8.3f kcal/mol" %(-dG/4.186))
# 1 kcal = 4.186 KJ

dG_off =  -12.132 kcal/mol
dG_on  =   12.132 kcal/mol


In [4]:
#===================================================================================================
# ROUX FORMULA
#===================================================================================================

thA = math.radians(thA_0)  # convert angle from degrees to radians --> math.sin() wants radians
thB = math.radians(thB_0)  # convert angle from degrees to radians --> math.sin() wants radians

Ft = r0 ** 2 * math.sin(thA) * ((2.0 * math.pi * K * T)**(3.0) / (K_dist * K_ang ** 2)) ** 0.5

Fr = math.sin(thB) * (2.0 * math.pi * K * T / K_rotate) ** 1.5 / (8 * math.pi ** 2)

In [5]:
dG = - K * T * math.log(Ft / V * Fr)
print (f"dG = {dG/4.186:.3f} kcal/mol")

dG = 12.132 kcal/mol
